In [292]:
import importlib

import plotly.graph_objects as go

import apollo.providers.price_data_provider as pdp
from apollo.providers.price_data_enhancer import PriceDataEnhancer
from apollo.providers.price_data_provider import PriceDataProvider
from apollo.settings import FREQUENCY, LONG_SIGNAL, SHORT_SIGNAL, TICKER
from apollo.utils.common import to_default_date_string

importlib.reload(pdp);

In [293]:
start_date = "2020-03-01"
end_date = "2020-06-01"

price_data_provider = PriceDataProvider()
price_data_enhancer = PriceDataEnhancer()

dataframe = price_data_provider.get_price_data(
    ticker=str(TICKER),
    frequency=str(FREQUENCY),
    start_date=start_date,
    end_date=end_date,
    max_period=False,
)

dataframe = price_data_enhancer.enhance_price_data(
    price_dataframe=dataframe,
    additional_data_enhancers=["VIX", "SP500 Futures"],
)

dataframe["pct_change"] = dataframe["adj close"].pct_change()
dataframe["vix_pct_change"] = dataframe["vix close"].pct_change()
dataframe["spf_pct_change"] = dataframe["spf close"].pct_change()

dataframe["vix_spf_pct_change_diff"] = (
    dataframe["vix_pct_change"] - dataframe["spf_pct_change"]
)

dataframe["prev_vix_spf_pct_change_diff"] = dataframe["vix_spf_pct_change_diff"].shift(
    1,
)

dataframe["prev_close"] = dataframe["adj close"].shift(1)

long = (dataframe["adj close"] > dataframe["prev_close"]) & (
    dataframe["vix_spf_pct_change_diff"] < dataframe["prev_vix_spf_pct_change_diff"]
)

dataframe.loc[long, "signal"] = LONG_SIGNAL

short = (dataframe["adj close"] < dataframe["prev_close"]) & (
    dataframe["vix_spf_pct_change_diff"] > dataframe["prev_vix_spf_pct_change_diff"]
)

dataframe.loc[short, "signal"] = SHORT_SIGNAL

In [294]:
value_1 = "adj close"
value_2 = "vix_pct_change"
value_3 = "spf_pct_change"
value_4 = "vix_spf_pct_change_diff"

x = dataframe.index.to_numpy()

y1 = dataframe[value_1].to_numpy()
y2 = dataframe[value_2].to_numpy()
y3 = dataframe[value_3].to_numpy()
y4 = dataframe[value_4].to_numpy()

trace1 = go.Scatter(x=x, y=y1, name=value_1, yaxis="y1")
trace2 = go.Scatter(x=x, y=y2, name=value_2, yaxis="y2")
trace3 = go.Scatter(x=x, y=y3, name=value_3, yaxis="y2")
trace4 = go.Scatter(x=x, y=y4, name=value_4, yaxis="y2")

# Plot title
title = (
    f"{dataframe.iloc[0]["ticker"]} | VIX | Futures"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(
    data=[
        # trace1,
        trace2,
        trace3,
        trace4,
    ],
    layout=layout,
)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)